In [5]:
a = array('d', [1, 2, 3])

In [6]:
a

array('d', [1.0, 2.0, 3.0])

In [7]:
reprlib.repr(a)

"array('d', [1.0, 2.0, 3.0])"

In [14]:
reprlib.repr(a)[reprlib.repr(a).find('['): -1]

'[1.0, 2.0, 3.0]'

In [16]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [18]:
v1 = Vector([3, 4, 5])

In [19]:
v1

Vector([3.0, 4.0, 5.0])

In [21]:
len(v1)

3

In [22]:
v1[0]

3.0

In [23]:
v1[-1]

5.0

In [24]:
v7 = Vector(range(7))

In [25]:
v7[1:4]

array('d', [1.0, 2.0, 3.0])

In [26]:
class MySeq:
    def __getitem__(self, index):
        return index

In [27]:
s = MySeq()

In [28]:
s[1]

1

In [29]:
s[1:4]

slice(1, 4, None)

In [30]:
s[1:4:2]

slice(1, 4, 2)

In [31]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [32]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [33]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [34]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [9]:
from array import array
import reprlib
import math
import numbers
import functools
import operator

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
#     def __getitem__(self, index):
#         return self._components[index]

    def __getitem__(self, index):
        cls = type(self)
        # 如果传入的参数是切片，就返回改切片生成的Vector类
        if isinstance(index, slice):
            return cls(self._components[index])
        # 如果传入的参数是数，就返回列表中的元素
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        # 否则就报错啦
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        # 创建一个生成器表达式，惰性计算各个分量的散列值
        # hashes = (hash(x) for x in self._components)
        # 这里换成map方法
        hashes = map(hash, self._components)
        
        # 把hashes提供给reduce函数，第三个参数0是初始值
        return functools.reduce(operator.xor, hashes, 0)
            
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [42]:
v7 = Vector(range(7))

In [43]:
v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [44]:
v7[-1]

6.0

In [45]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [46]:
v7[-1:]

Vector([6.0])

In [47]:
v7[1,2]

TypeError: Vector indices must be integers

In [2]:
v = Vector((range(5)))

In [3]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [4]:
v.x

0.0

In [5]:
v.y

1.0

In [6]:
v.x = 10

In [7]:
v.x

10

In [8]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [10]:
import functools
functools.reduce(lambda a, b: a * b, range(1, 6))

120